# DLA for Z2 lattice gauge theory Hamiltonian variational ansatz (dense implementation)

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import h5py
from qiskit.quantum_info import SparsePauliOp
from fastdla.dla_dense import generate_dla, is_independent
from fastdla.dladim_dense import count_dla_dim
from fastdla.ansatze.z2lgt_hva import z2lgt_hva_generators, z2lgt_dense_projector
import jax
jax.config.update('jax_enable_x64', True)
import jax.numpy as jnp


In [2]:
data_file = 'z2lgt_hva_dla_dense.h5'

## 2 Fermions

In [3]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
charge = 0

In [ ]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full


In [ ]:
# Projector
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge, npmod=jnp)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')
csrs = [SparsePauliOp(op.paulis, op.coeffs).to_matrix(sparse=True) for op in generators_full]
generators = [dense_projector @ mat.dot(dense_projector.conjugate().T) for mat in csrs]
generators = [op / np.sqrt(np.abs(np.trace(op.conjugate().T @ op))) for op in generators]

In [ ]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

In [ ]:
# Compute the DLA of the subspace
dladim = count_dla_dim(generators, verbosity=2)
print(f'Subspace DLA dimension is {dladim}')

In [8]:
with h5py.File(data_file, 'a') as out:
    if 'nf=2' not in out:
        nf_group = out.create_group('nf=2')
        for gname, oplist in [
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('ops', data=np.array(oplist))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)

## 3 Fermions

In [9]:
num_fermions = 3
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1]
charge = 0

In [ ]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

In [ ]:
# Projector
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')
csrs = [SparsePauliOp(op.paulis, op.coeffs).to_matrix(sparse=True) for op in generators_full]
generators = [dense_projector @ mat.dot(dense_projector.conjugate().T) for mat in csrs]
generators = [op / np.sqrt(np.abs(np.trace(op.conjugate().T @ op))) for op in generators]

In [ ]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

In [ ]:
# Compute the DLA of the subspace
dladim = count_dla_dim(generators, verbosity=2)
print(f'Subspace DLA dimension is {dladim}')

In [14]:
with h5py.File(data_file, 'a') as out:
    if 'nf=3' not in out:
        nf_group = out.create_group('nf=3')
        for gname, oplist in [
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('ops', data=np.array(oplist))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)